# CDL Block Visualization Tutorial

This notebook demonstrates the visualization capabilities of Python CDL. Learn how to create beautiful, publication-quality diagrams of your control systems.

## What You'll Learn

1. **Visualize Elementary Blocks** - See inputs, outputs, equations, and parameters
2. **Visualize Composite Blocks** - Understand hierarchical structures and connections
3. **Customization** - Control styles, colors, and detail levels
4. **Multiple Backends** - Use Matplotlib or Graphviz renderers
5. **Save to Files** - Export diagrams in PNG, PDF, SVG formats

## Use Cases

- 📊 **Documentation** - Generate diagrams for technical documentation
- 🎓 **Education** - Teach control system concepts visually
- 🔍 **Debugging** - Understand complex system structures
- 📝 **Presentations** - Create professional slides and reports
- ✅ **Verification** - Verify system architecture before deployment

---

## Setup: Import Libraries

In [ ]:
# Standard library
import json
import sys
from pathlib import Path

# Add project to path
project_root = Path.cwd().parent if Path.cwd().name == 'examples' else Path.cwd()
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Import Python CDL
from python_cdl.models.blocks import Block, CompositeBlock
from python_cdl.models.connectors import RealInput, RealOutput
from python_cdl.models.parameters import Parameter
from python_cdl.models.equations import Equation
from python_cdl.models.connections import Connection
from python_cdl.parser.json_parser import CDLParser
from python_cdl.visualization import BlockVisualizer, VisualizationStyle

import matplotlib.pyplot as plt
%matplotlib inline

print("✓ All imports successful!")
print(f"✓ Python CDL loaded from: {project_root}")

---

## Part 1: Visualizing Elementary Blocks

Let's start by visualizing a simple P-controller.

In [ ]:
# Create a P-controller
p_controller = Block(
    name="PController",
    block_type="elementary",
    parameters=[
        Parameter(name="k", type="Real", value=2.0, description="Proportional gain"),
    ],
    inputs=[
        RealInput(name="u_s", description="Setpoint", unit="degC"),
        RealInput(name="u_m", description="Measurement", unit="degC"),
    ],
    outputs=[
        RealOutput(name="y", description="Control output", unit="1"),
    ],
    equations=[
        Equation(lhs="e", rhs="u_s - u_m"),
        Equation(lhs="y", rhs="k * e"),
    ]
)

print(f"Created block: {p_controller.name}")
print(f"  Inputs: {[inp.name for inp in p_controller.inputs]}")
print(f"  Outputs: {[out.name for out in p_controller.outputs]}")
print(f"  Equations: {len(p_controller.equations)}")

### Basic Visualization

Use `BlockVisualizer` to create a diagram:

In [ ]:
# Create visualizer
viz = BlockVisualizer()

# Render the block
fig = viz.render(p_controller)
plt.show()

print("\n📊 The diagram shows:")
print("  • Blue boxes = Inputs (u_s, u_m)")
print("  • Green box = P-controller block with equation")
print("  • Yellow box = Output (y)")
print("  • Arrows = Data flow")

### Customizing Visualization Style

Control the level of detail shown:

In [ ]:
# Show different styles side by side
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

styles = [VisualizationStyle.DETAILED, VisualizationStyle.SIMPLE, VisualizationStyle.COMPACT]
titles = ["Detailed (equations shown)", "Simple (block names only)", "Compact (minimal)"]

for ax, style, title in zip(axes, styles, titles):
    viz = BlockVisualizer(style=style)
    # Note: In practice, you'd create separate figures
    # This is for demonstration
    ax.set_title(title, fontsize=12, fontweight='bold')
    ax.axis('off')

plt.tight_layout()

# Show detailed style
print("\nVisualization Styles:")
print("  • DETAILED: Shows equations, parameters, types")
print("  • SIMPLE: Shows block names and connections only")
print("  • COMPACT: Minimal view for high-level overview")

In [ ]:
# Actually render with different styles
for style in [VisualizationStyle.DETAILED, VisualizationStyle.SIMPLE]:
    print(f"\n{style.value.upper()} Style:")
    viz = BlockVisualizer(style=style)
    fig = viz.render(p_controller)
    plt.show()

---

## Part 2: Visualizing Composite Blocks

Now let's visualize a more complex composite block with multiple child blocks.

In [ ]:
# Load the P-controller with limiter example
parser = CDLParser()

with open(project_root / 'examples' / 'p_controller_limiter.json') as f:
    block_data = json.load(f)

composite_block = parser.parse_block(block_data)

print(f"Loaded composite block: {composite_block.name}")
print(f"\nStructure:")
print(f"  Inputs: {[inp.name for inp in composite_block.inputs]}")
print(f"  Child blocks: {[b.name for b in composite_block.blocks]}")
print(f"  Outputs: {[out.name for out in composite_block.outputs]}")
print(f"  Connections: {len(composite_block.connections)}")

In [ ]:
# Visualize the composite block
viz = BlockVisualizer(style=VisualizationStyle.DETAILED)
fig = viz.render(composite_block)
plt.show()

print("\n📊 The composite block diagram shows:")
print("  • External inputs (e, yMax) on the left")
print("  • Internal blocks (gain, minValue) in the middle")
print("  • External output (y) on the right")
print("  • Connections showing data flow between all components")
print("\n🎯 Key Insight:")
print("  The visualization makes it easy to understand that:")
print("  1. Error 'e' is multiplied by gain (k=5.0)")
print("  2. Result is limited by min(gain_output, yMax)")
print("  3. Final limited value becomes output 'y'")

---

## Part 3: Building and Visualizing Custom Systems

Let's build a control system programmatically and visualize it.

In [ ]:
# Create a temperature control system with PI controller and limiter

# 1. PI Controller block
pi_controller = Block(
    name="PIController",
    block_type="PID",
    parameters=[
        Parameter(name="kp", type="Real", value=0.5),
        Parameter(name="Ti", type="Real", value=60.0, unit="s"),
    ],
    inputs=[
        RealInput(name="setpoint", unit="K"),
        RealInput(name="measurement", unit="K"),
    ],
    outputs=[
        RealOutput(name="control_signal", unit="1"),
    ]
)

# 2. Output limiter block
limiter = Block(
    name="OutputLimiter",
    block_type="Limiter",
    parameters=[
        Parameter(name="uMin", type="Real", value=0.0),
        Parameter(name="uMax", type="Real", value=1.0),
    ],
    inputs=[RealInput(name="u", unit="1")],
    outputs=[RealOutput(name="y", unit="1", min=0.0, max=1.0)]
)

# 3. Composite system
temp_control_system = CompositeBlock(
    name="TemperatureControlSystem",
    block_type="composite",
    description="PI temperature control with output limiting",
    inputs=[
        RealInput(name="temperature_setpoint", unit="K"),
        RealInput(name="temperature_measurement", unit="K"),
    ],
    outputs=[
        RealOutput(name="valve_position", unit="1", min=0.0, max=1.0),
    ],
    blocks=[pi_controller, limiter],
    connections=[
        Connection(
            from_block="temperature_setpoint",
            from_output="",
            to_block="PIController",
            to_input="setpoint",
            description="Setpoint to controller"
        ),
        Connection(
            from_block="temperature_measurement",
            from_output="",
            to_block="PIController",
            to_input="measurement",
            description="Measurement to controller"
        ),
        Connection(
            from_block="PIController",
            from_output="control_signal",
            to_block="OutputLimiter",
            to_input="u",
            description="Controller to limiter"
        ),
        Connection(
            from_block="OutputLimiter",
            from_output="y",
            to_block="valve_position",
            to_input="",
            description="Limited output to valve"
        ),
    ]
)

print("✓ Created temperature control system")
print(f"  {len(temp_control_system.inputs)} inputs")
print(f"  {len(temp_control_system.blocks)} internal blocks")
print(f"  {len(temp_control_system.connections)} connections")
print(f"  {len(temp_control_system.outputs)} outputs")

In [ ]:
# Visualize the custom system
viz = BlockVisualizer(style=VisualizationStyle.DETAILED)
fig = viz.render(temp_control_system)
plt.show()

print("\n💡 This diagram is perfect for:")
print("  • Documentation - Include in technical specifications")
print("  • Code reviews - Verify system architecture")
print("  • Presentations - Explain control logic visually")
print("  • Debugging - Trace signal flow through system")

---

## Part 4: Customization and Styling

Fine-tune the appearance of your diagrams.

In [ ]:
# Customize colors and appearance
from python_cdl.visualization import MatplotlibRenderer

# Create custom renderer with different colors
custom_renderer = MatplotlibRenderer(
    style=VisualizationStyle.DETAILED,
    show_parameters=True,
    show_equations=True,
    figsize=(16, 10),
    input_color='#E3F2FD',      # Light blue
    output_color='#FFF9C4',     # Light yellow
    elementary_color='#C8E6C9', # Light green
    composite_color='#FFCCBC',  # Light orange
    connection_color='#424242'  # Dark gray
)

fig = custom_renderer.render(temp_control_system)
plt.show()

print("\n🎨 Customization options:")
print("  • Colors for inputs, outputs, and blocks")
print("  • Figure size and DPI")
print("  • Show/hide parameters, equations, types")
print("  • Connection arrow styles")

### Toggle Details On/Off

Control what information is displayed:

In [ ]:
# Compare: Full details vs. minimal
print("Full details (parameters + equations + types):")
viz_full = BlockVisualizer(
    style=VisualizationStyle.DETAILED,
    show_parameters=True,
    show_equations=True,
    show_types=True
)
fig = viz_full.render(p_controller)
plt.show()

print("\nMinimal (names only):")
viz_minimal = BlockVisualizer(
    style=VisualizationStyle.SIMPLE,
    show_parameters=False,
    show_equations=False,
    show_types=False
)
fig = viz_minimal.render(p_controller)
plt.show()

---

## Part 5: Saving Diagrams

Export visualizations to files for use in documentation.

In [ ]:
import os

# Create output directory
output_dir = Path("visualization_output")
output_dir.mkdir(exist_ok=True)

# Save as PNG (for web/docs)
viz = BlockVisualizer(style=VisualizationStyle.DETAILED)
viz.render(temp_control_system, output_file=str(output_dir / "temp_control.png"))
print(f"✓ Saved PNG: {output_dir / 'temp_control.png'}")

# Save as PDF (for publications)
viz.render(temp_control_system, output_file=str(output_dir / "temp_control.pdf"))
print(f"✓ Saved PDF: {output_dir / 'temp_control.pdf'}")

# Save elementary block too
viz.render(p_controller, output_file=str(output_dir / "p_controller.png"))
print(f"✓ Saved PNG: {output_dir / 'p_controller.png'}")

print("\n📁 Files saved to visualization_output/")
print("  • Use PNG for web pages and documentation")
print("  • Use PDF for high-quality prints and publications")
print("  • Use SVG for scalable vector graphics (with graphviz backend)")

---

## Part 6: Advanced - Visualizing Nested Composites

Create and visualize multi-level hierarchical systems.

In [ ]:
# Create a building control system with multiple subsystems

# This would be a zone controller (composite)
zone_controller = CompositeBlock(
    name="ZoneController",
    block_type="composite",
    inputs=[
        RealInput(name="zone_temp", unit="K"),
        RealInput(name="setpoint", unit="K"),
    ],
    outputs=[
        RealOutput(name="damper_cmd", unit="1"),
        RealOutput(name="reheat_cmd", unit="1"),
    ],
    blocks=[],  # Would contain actual control blocks
    connections=[]
)

# Building system contains multiple zone controllers
building_system = CompositeBlock(
    name="BuildingControlSystem",
    block_type="composite",
    description="Multi-zone building HVAC control",
    inputs=[
        RealInput(name="outdoor_temp", unit="K"),
        RealInput(name="occupancy", unit="1"),
    ],
    outputs=[
        RealOutput(name="fan_speed", unit="1"),
        RealOutput(name="cooling_valve", unit="1"),
    ],
    blocks=[zone_controller],
    connections=[]
)

print("✓ Created nested composite system")
print("  Top level: Building control")
print("  Second level: Zone controllers")
print("  (In real systems, zone controllers would contain more blocks)")

---

## Summary

### What We Learned

✅ **Basic visualization** - Render blocks with one line of code  
✅ **Customization** - Control styles, colors, and detail levels  
✅ **Composite blocks** - Visualize hierarchical systems  
✅ **File export** - Save diagrams in multiple formats  
✅ **Publication quality** - Create professional diagrams  

### API Summary

```python
from python_cdl.visualization import BlockVisualizer, VisualizationStyle

# Basic usage
viz = BlockVisualizer()
fig = viz.render(block)
plt.show()

# Customized
viz = BlockVisualizer(
    style=VisualizationStyle.DETAILED,
    show_parameters=True,
    show_equations=True
)

# Save to file
viz.render(block, output_file="diagram.png")
```

### Key Features

1. **Multiple styles** - Detailed, Simple, Compact
2. **Two backends** - Matplotlib (default), Graphviz (optional)
3. **Customizable** - Colors, sizes, labels
4. **Export formats** - PNG, PDF, SVG
5. **Production ready** - Publication-quality output

### Use Cases

- 📚 **Technical documentation** - Auto-generate architecture diagrams
- 🎓 **Educational materials** - Teach control concepts visually
- 🔧 **System design** - Visualize before implementation
- 🐛 **Debugging** - Understand complex systems
- ✅ **Verification** - Verify architecture visually
- 📊 **Presentations** - Professional slides and reports

---

*Visualization module - Making CDL systems visual and understandable!*